In [1]:
!pip install pyomo -q
!pip install mistralai -q
!wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00


In [2]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [3]:
API_KEY = ''

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 3
MODEL_TEMPERATURE = 0.7

SYSTEM_PROMPT_1 = """Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."""
SYSTEM_PROMPT_2 = """Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."""
PROBLEM_DESCRIPTION = """A buyer needs to acquire 239,600,480 units of a product and is considering bids from five suppliers, labeled A through E, each of whom can only supply a portion of the total required amount.
Each vendor has proposed different pricing structures, incorporating both setup fees and variable unit costs that change based on the quantity ordered.

The buyer's objective is to allocate the order among these suppliers to minimize overall costs, accounting for both setup and unit costs.

Vendor A offers a set up cost of $3855.34 and a unit cost of $61.150 per thousand of units.
Vendor A can supply up to 33 million units.

Vendor B offers a set up cost of $125,804.84 if purchasing between 22,000,000-70,000,000 units from vendor B with a unit cost of $68.099 per thousand units.
If purchasing between 70,000,001-100,000,000 units from vendor B, the set up cost increases to $269304.84 and the unit cost sinks to $66.049 per thousand units.
If purchasing between 100,000,001-150,000,000 units from vendor B, the unit cost per thousand units further decreases to $64.099, but the set up cost increases to $464304.84.
If purchasing between 150,000,001 and 160,000,000 units from vendor B, the unit cost is $62.119 per thousand units and the set up cost equals $761304.84.

Vendor C offers set up costs of $13,456.00 and a unit cost of $62.019 per thousand units.
Vendor C can supply up to 165.6 million units. Vendor D offers set up costs of $6,583.98 and a unit cost of $72.488 for a set of thousand units.

Vendor D can supply up to 12 million units at a price of $72.488 per thousand units and with a set up cost of $6583.98.

Vendor E offers free set up if purchasing between 0 and 42 million units of vendor E with a unit price of $70.150 per thousand units.
If purchasing between 42,000,001 and 77 million units from vendor E, the unit cost starts at $68.150 per thousand units, but with every one million units purchased the price decreases at a rate of 0.05 percent. An additional set up cost of $84000 will be charged as well.

Note that zero units may be purchased from vendor B: otherwise no positive number of units less than 22,000,000 may be purchased."""


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-06-14 14:20:35.167341


## Step 1 - Generate Mathematical Formulation

In [4]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION)
]

In [5]:
response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_1_text = response_1.choices[0].message.content

In [6]:
Markdown(response_1_text)

This problem can be formulated as a Mixed Integer Linear Programming (MILP) model. Here are the parameters, decision variables, objective function, and constraints:

Parameters:

1. Total demand (D): 239,600,480 units
2. Maximum supply from vendor A (A_max): 33,000,000 units
3. Setup cost for vendor A (A_setup): $3855.34
4. Unit cost for vendor A (A_unit): $61.150 per thousand units
5. Setup cost for vendor B (B_setup1): $125,804.84
6. Unit cost for vendor B (B_unit1): $68.099 per thousand units
7. Setup cost for vendor B (B_setup2): $269,304.84
8. Unit cost for vendor B (B_unit2): $66.049 per thousand units
9. Setup cost for vendor B (B_setup3): $464,304.84
10. Unit cost for vendor B (B_unit3): $64.099 per thousand units
11. Setup cost for vendor B (B_setup4): $761,304.84
12. Unit cost for vendor B (B_unit4): $62.119 per thousand units
13. Maximum supply from vendor B (B_max): 160,000,000 units
14. Minimum units for vendor B (B_min): 22,000,000 units
15. Setup cost for vendor C (C_setup): $13,456.00
16. Unit cost for vendor C (C_unit): $62.019 per thousand units
17. Maximum supply from vendor C (C_max): 165,600,000 units
18. Setup cost for vendor D (D_setup): $6,583.98
19. Unit cost for vendor D (D_unit): $72.488 per thousand units
20. Maximum supply from vendor D (D_max): 12,000,000 units
21. Setup cost for vendor E (E_setup): $84,000.00
22. Unit cost for vendor E (E_unit1): $70.150 per thousand units
23. Unit cost for vendor E (E_unit2): $68.150 per thousand units
24. Additional setup cost for vendor E (E_add_setup): $84,000.00
25. Maximum supply from vendor E (E_max): 77,000,000 units
26. Minimum units for vendor E (E_min): 42,000,001 units

Decision Variables:

1. Number of units from vendor A (x_A): integer
2. Number of units from vendor B (x_B): integer
3. Number of units from vendor C (x_C): integer
4. Number of units from vendor D (x_D): integer
5. Number of units from vendor E (x_E): integer

Objective Function:

Minimize: A_setup * y_A + A_unit * x_A/1000 + B_setup1 * y_B1 + B_setup2 * y_B2 + B_setup3 * y_B3 + B_setup4 * y_B4 + B_unit1 * x_B1/1000 + B_unit2 * x_B2/1000 + B_unit3 * x_B3/1000 + B_unit4 * x_B4/1000 + C_setup * y_C + C_unit * x_C/1000 + D_setup * y_D + D_unit * x_D/1000 + E_setup * y_E + E_unit1 * x_E1/1000 + E_unit2 * x_E2/1000 + E_add_setup * y_E

Constraints:

1. x_A + x_B + x_C + x_D + x_E = D
2. x_A <= A_max * y_A
3. x_B1 + x_B2 + x_B3 + x_B4 = x_B
4. x_B1 <= 48,000,000 * y_B1
5. x_B2 <= 30,000,000 * y_B2
6. x_B3 <= 50,000,000 * y_B3
7. x_B4 <= 10,000,000 * y_B4
8. x_B >= B_min * (y_B1 + y_B2 + y_B3 + y_B4)
9. x_C <= C_max * y_C
10. x_D <= D_max * y_D
11. x_E1 + x_E2 = x_E
12. x_E1 <= 42,000,000 * y_E
13. x_E2 <= 35,000,000 * y_E
14. x_E2 >= E_min * y_E
15. y_A, y_B1, y_B2, y_B3, y_B4, y_C, y_D, y_E ∈ {0, 1}
16. x_A, x_B, x_C, x_D, x_E1, x_E2 ∈ ℤ

In [7]:
print(response_1_text)

This problem can be formulated as a Mixed Integer Linear Programming (MILP) model. Here are the parameters, decision variables, objective function, and constraints:

Parameters:

1. Total demand (D): 239,600,480 units
2. Maximum supply from vendor A (A_max): 33,000,000 units
3. Setup cost for vendor A (A_setup): $3855.34
4. Unit cost for vendor A (A_unit): $61.150 per thousand units
5. Setup cost for vendor B (B_setup1): $125,804.84
6. Unit cost for vendor B (B_unit1): $68.099 per thousand units
7. Setup cost for vendor B (B_setup2): $269,304.84
8. Unit cost for vendor B (B_unit2): $66.049 per thousand units
9. Setup cost for vendor B (B_setup3): $464,304.84
10. Unit cost for vendor B (B_unit3): $64.099 per thousand units
11. Setup cost for vendor B (B_setup4): $761,304.84
12. Unit cost for vendor B (B_unit4): $62.119 per thousand units
13. Maximum supply from vendor B (B_max): 160,000,000 units
14. Minimum units for vendor B (B_min): 22,000,000 units
15. Setup cost for vendor C (C_set

## Step 2 - Generate the Pyomo Code

In [8]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_1_text)
]

In [9]:
response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [10]:
Markdown(response_2_text)

Here is the Python code using Pyomo to solve the given Mixed Integer Linear Programming (MILP) problem:

```python
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Define the parameters
model.D = 239600480
model.A_max = 33000000
model.A_setup = 3855.34
model.A_unit = 61.150
model.B_max = 160000000
model.B_min = 22000000
model.B_setup1 = 125804.84
model.B_unit1 = 68.099
model.B_setup2 = 269304.84
model.B_unit2 = 66.049
model.B_setup3 = 464304.84
model.B_unit3 = 64.099
model.B_setup4 = 761304.84
model.B_unit4 = 62.119
model.C_setup = 13456.00
model.C_unit = 62.019
model.C_max = 165600000
model.D_setup = 6583.98
model.D_unit = 72.488
model.D_max = 12000000
model.E_setup = 84000.00
model.E_unit1 = 70.150
model.E_unit2 = 68.150
model.E_add_setup = 84000.00
model.E_max = 77000000
model.E_min = 42000001

# Define the decision variables
model.x_A = pyo.Var(within=pyo.PositiveIntegers)
model.x_B = pyo.Var(within=py.PositiveIntegers)
model.x_C = pyo.Var(within=pyo.PositiveIntegers)
model.x_D = pyo.Var(within=pyo.PositiveIntegers)
model.x_E = pyo.Var(within=pyo.PositiveIntegers)

model.y_A = pyo.Var(within=pyo.Binary)
model.y_B1 = pyo.Var(within=pyo.Binary)
model.y_B2 = pyo.Var(within=pyo.Binary)
model.y_B3 = pyo.Var(within=pyo.Binary)
model.y_B4 = pyo.Var(within=pyo.Binary)
model.y_C = pyo.Var(within=pyo.Binary)
model.y_D = pyo.Var(within=pyo.Binary)
model.y_E = pyo.Var(within=pyo.Binary)

# Define the objective function
model.OBJ = pyo.Objective(
    expr = (
        model.A_setup * model.y_A
        + model.A_unit * model.x_A / 1000
        + model.B_setup1 * model.y_B1
        + model.B_setup2 * model.y_B2
        + model.B_setup3 * model.y_B3
        + model.B_setup4 * model.y_B4
        + model.B_unit1 * model.x_B / 1000
        + model.B_unit2 * model.x_B / 1000
        + model.B_unit3 * model.x_B / 1000
        + model.B_unit4 * model.x_B / 1000
        + model.C_setup * model.y_C
        + model.C_unit * model.x_C / 1000
        + model.D_setup * model.y_D
        + model.D_unit * model.x_D / 1000
        + model.E_setup * model.y_E
        + model.E_unit1 * model.x_E / 1000
        + model.E_unit2 * model.x_E / 1000
        + model.E_add_setup * model.y_E
    ),
    sense=pyo.minimize
)

# Define the constraints
model.DEMAND_CONSTRAINT = pyo.Constraint(
    expr = model.x_A + model.x_B + model.x_C + model.x_D + model.x_E == model.D
)
model.VENDOR_A_CONSTRAINT = pyo.Constraint(
    expr = model.x_A <= model.A_max * model.y_A
)
model.VENDOR_B_CONSTRAINT = pyo.Constraint(
    expr = model.x_B * (model.y_B1 + model.y_B2 + model.y_B3 + model.y_B4) >= model.B_min
)
model.VENDOR_B1_CONSTRAINT = pyo.Constraint(
    expr = model.x_B * model.y_B1 <= 48000000 * model.y_B1
)
model.VENDOR_B2_CONSTRAINT = pyo.Constraint(
    expr = model.x_B * model.y_B2 <= 30000000 * model.y_B2
)
model.VENDOR_B3_CONSTRAINT = pyo.Constraint(
    expr = model.x_B * model.y_B3 <= 50000000 * model.y_B3
)
model.VENDOR_B4_CONSTRAINT = pyo.Constraint(
    expr = model.x_B * model.y_B4 <= 10000000 * model.y_B4
)
model.VENDOR_C_CONSTRAINT = pyo.Constraint(
    expr = model.x_C <= model.C_max * model.y_C
)
model.VENDOR_D_CONSTRAINT = pyo.Constraint(
    expr = model.x_D <= model.D_max * model.y_D
)
model.VENDOR_E_CONSTRAINT = pyo.Constraint(
    expr = model.x_E * model.y_E >= model.E_min * model.y_E
)

# Solve the model
solver = pyo.SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print('Total Cost: ', pyo.value(model.OBJ))
print('Number of units from vendor A: ', pyo.value(model.x_A))
print('Number of units from vendor B: ', pyo.value(model.x_B))
print('Number of units from vendor C: ', pyo.value(model.x_C))
print('Number of units from vendor D: ', pyo.value(model.x_D))
print('Number of units from vendor E: ', pyo.value(model.x_E))
```

In [11]:
print(response_2_text)

Here is the Python code using Pyomo to solve the given Mixed Integer Linear Programming (MILP) problem:

```python
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Define the parameters
model.D = 239600480
model.A_max = 33000000
model.A_setup = 3855.34
model.A_unit = 61.150
model.B_max = 160000000
model.B_min = 22000000
model.B_setup1 = 125804.84
model.B_unit1 = 68.099
model.B_setup2 = 269304.84
model.B_unit2 = 66.049
model.B_setup3 = 464304.84
model.B_unit3 = 64.099
model.B_setup4 = 761304.84
model.B_unit4 = 62.119
model.C_setup = 13456.00
model.C_unit = 62.019
model.C_max = 165600000
model.D_setup = 6583.98
model.D_unit = 72.488
model.D_max = 12000000
model.E_setup = 84000.00
model.E_unit1 = 70.150
model.E_unit2 = 68.150
model.E_add_setup = 84000.00
model.E_max = 77000000
model.E_min = 42000001

# Define the decision variables
model.x_A = pyo.Var(within=pyo.PositiveIntegers)
model.x_B = pyo.Var(within=py.PositiveIntegers)
model.x_C = pyo.Var(within=pyo.P

### Code Executability

In [12]:
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Define the parameters
model.D = 239600480
model.A_max = 33000000
model.A_setup = 3855.34
model.A_unit = 61.150
model.B_max = 160000000
model.B_min = 22000000
model.B_setup1 = 125804.84
model.B_unit1 = 68.099
model.B_setup2 = 269304.84
model.B_unit2 = 66.049
model.B_setup3 = 464304.84
model.B_unit3 = 64.099
model.B_setup4 = 761304.84
model.B_unit4 = 62.119
model.C_setup = 13456.00
model.C_unit = 62.019
model.C_max = 165600000
model.D_setup = 6583.98
model.D_unit = 72.488
model.D_max = 12000000
model.E_setup = 84000.00
model.E_unit1 = 70.150
model.E_unit2 = 68.150
model.E_add_setup = 84000.00
model.E_max = 77000000
model.E_min = 42000001

# Define the decision variables
model.x_A = pyo.Var(within=pyo.PositiveIntegers)
model.x_B = pyo.Var(within=py.PositiveIntegers)
model.x_C = pyo.Var(within=pyo.PositiveIntegers)
model.x_D = pyo.Var(within=pyo.PositiveIntegers)
model.x_E = pyo.Var(within=pyo.PositiveIntegers)

model.y_A = pyo.Var(within=pyo.Binary)
model.y_B1 = pyo.Var(within=pyo.Binary)
model.y_B2 = pyo.Var(within=pyo.Binary)
model.y_B3 = pyo.Var(within=pyo.Binary)
model.y_B4 = pyo.Var(within=pyo.Binary)
model.y_C = pyo.Var(within=pyo.Binary)
model.y_D = pyo.Var(within=pyo.Binary)
model.y_E = pyo.Var(within=pyo.Binary)

# Define the objective function
model.OBJ = pyo.Objective(
    expr = (
        model.A_setup * model.y_A
        + model.A_unit * model.x_A / 1000
        + model.B_setup1 * model.y_B1
        + model.B_setup2 * model.y_B2
        + model.B_setup3 * model.y_B3
        + model.B_setup4 * model.y_B4
        + model.B_unit1 * model.x_B / 1000
        + model.B_unit2 * model.x_B / 1000
        + model.B_unit3 * model.x_B / 1000
        + model.B_unit4 * model.x_B / 1000
        + model.C_setup * model.y_C
        + model.C_unit * model.x_C / 1000
        + model.D_setup * model.y_D
        + model.D_unit * model.x_D / 1000
        + model.E_setup * model.y_E
        + model.E_unit1 * model.x_E / 1000
        + model.E_unit2 * model.x_E / 1000
        + model.E_add_setup * model.y_E
    ),
    sense=pyo.minimize
)

# Define the constraints
model.DEMAND_CONSTRAINT = pyo.Constraint(
    expr = model.x_A + model.x_B + model.x_C + model.x_D + model.x_E == model.D
)
model.VENDOR_A_CONSTRAINT = pyo.Constraint(
    expr = model.x_A <= model.A_max * model.y_A
)
model.VENDOR_B_CONSTRAINT = pyo.Constraint(
    expr = model.x_B * (model.y_B1 + model.y_B2 + model.y_B3 + model.y_B4) >= model.B_min
)
model.VENDOR_B1_CONSTRAINT = pyo.Constraint(
    expr = model.x_B * model.y_B1 <= 48000000 * model.y_B1
)
model.VENDOR_B2_CONSTRAINT = pyo.Constraint(
    expr = model.x_B * model.y_B2 <= 30000000 * model.y_B2
)
model.VENDOR_B3_CONSTRAINT = pyo.Constraint(
    expr = model.x_B * model.y_B3 <= 50000000 * model.y_B3
)
model.VENDOR_B4_CONSTRAINT = pyo.Constraint(
    expr = model.x_B * model.y_B4 <= 10000000 * model.y_B4
)
model.VENDOR_C_CONSTRAINT = pyo.Constraint(
    expr = model.x_C <= model.C_max * model.y_C
)
model.VENDOR_D_CONSTRAINT = pyo.Constraint(
    expr = model.x_D <= model.D_max * model.y_D
)
model.VENDOR_E_CONSTRAINT = pyo.Constraint(
    expr = model.x_E * model.y_E >= model.E_min * model.y_E
)

# Solve the model
solver = pyo.SolverFactory('ipopt')
results = solver.solve(model)

# Print the results
print('Total Cost: ', pyo.value(model.OBJ))
print('Number of units from vendor A: ', pyo.value(model.x_A))
print('Number of units from vendor B: ', pyo.value(model.x_B))
print('Number of units from vendor C: ', pyo.value(model.x_C))
print('Number of units from vendor D: ', pyo.value(model.x_D))
print('Number of units from vendor E: ', pyo.value(model.x_E))

NameError: name 'py' is not defined

### Solution Correctness

In [13]:
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Define the parameters
model.D = 239600480
model.A_max = 33000000
model.A_setup = 3855.34
model.A_unit = 61.150
model.B_max = 160000000
model.B_min = 22000000
model.B_setup1 = 125804.84
model.B_unit1 = 68.099
model.B_setup2 = 269304.84
model.B_unit2 = 66.049
model.B_setup3 = 464304.84
model.B_unit3 = 64.099
model.B_setup4 = 761304.84
model.B_unit4 = 62.119
model.C_setup = 13456.00
model.C_unit = 62.019
model.C_max = 165600000
model.D_setup = 6583.98
model.D_unit = 72.488
model.D_max = 12000000
model.E_setup = 84000.00
model.E_unit1 = 70.150
model.E_unit2 = 68.150
model.E_add_setup = 84000.00
model.E_max = 77000000
model.E_min = 42000001

# Define the decision variables
model.x_A = pyo.Var(within=pyo.PositiveIntegers)
model.x_B = pyo.Var(within=pyo.PositiveIntegers)
model.x_C = pyo.Var(within=pyo.PositiveIntegers)
model.x_D = pyo.Var(within=pyo.PositiveIntegers)
model.x_E = pyo.Var(within=pyo.PositiveIntegers)

model.y_A = pyo.Var(within=pyo.Binary)
model.y_B1 = pyo.Var(within=pyo.Binary)
model.y_B2 = pyo.Var(within=pyo.Binary)
model.y_B3 = pyo.Var(within=pyo.Binary)
model.y_B4 = pyo.Var(within=pyo.Binary)
model.y_C = pyo.Var(within=pyo.Binary)
model.y_D = pyo.Var(within=pyo.Binary)
model.y_E = pyo.Var(within=pyo.Binary)

# Define the objective function
model.OBJ = pyo.Objective(
    expr = (
        model.A_setup * model.y_A
        + model.A_unit * model.x_A / 1000
        + model.B_setup1 * model.y_B1
        + model.B_setup2 * model.y_B2
        + model.B_setup3 * model.y_B3
        + model.B_setup4 * model.y_B4
        + model.B_unit1 * model.x_B / 1000
        + model.B_unit2 * model.x_B / 1000
        + model.B_unit3 * model.x_B / 1000
        + model.B_unit4 * model.x_B / 1000
        + model.C_setup * model.y_C
        + model.C_unit * model.x_C / 1000
        + model.D_setup * model.y_D
        + model.D_unit * model.x_D / 1000
        + model.E_setup * model.y_E
        + model.E_unit1 * model.x_E / 1000
        + model.E_unit2 * model.x_E / 1000
        + model.E_add_setup * model.y_E
    ),
    sense=pyo.minimize
)

# Define the constraints
model.DEMAND_CONSTRAINT = pyo.Constraint(
    expr = model.x_A + model.x_B + model.x_C + model.x_D + model.x_E == model.D
)
model.VENDOR_A_CONSTRAINT = pyo.Constraint(
    expr = model.x_A <= model.A_max * model.y_A
)
model.VENDOR_B_CONSTRAINT = pyo.Constraint(
    expr = model.x_B * (model.y_B1 + model.y_B2 + model.y_B3 + model.y_B4) >= model.B_min
)
model.VENDOR_B1_CONSTRAINT = pyo.Constraint(
    expr = model.x_B * model.y_B1 <= 48000000 * model.y_B1
)
model.VENDOR_B2_CONSTRAINT = pyo.Constraint(
    expr = model.x_B * model.y_B2 <= 30000000 * model.y_B2
)
model.VENDOR_B3_CONSTRAINT = pyo.Constraint(
    expr = model.x_B * model.y_B3 <= 50000000 * model.y_B3
)
model.VENDOR_B4_CONSTRAINT = pyo.Constraint(
    expr = model.x_B * model.y_B4 <= 10000000 * model.y_B4
)
model.VENDOR_C_CONSTRAINT = pyo.Constraint(
    expr = model.x_C <= model.C_max * model.y_C
)
model.VENDOR_D_CONSTRAINT = pyo.Constraint(
    expr = model.x_D <= model.D_max * model.y_D
)
model.VENDOR_E_CONSTRAINT = pyo.Constraint(
    expr = model.x_E * model.y_E >= model.E_min * model.y_E
)

# Solve the model
solver = pyo.SolverFactory('ipopt')
results = solver.solve(model)

# Print the results
print('Total Cost: ', pyo.value(model.OBJ))
print('Number of units from vendor A: ', pyo.value(model.x_A))
print('Number of units from vendor B: ', pyo.value(model.x_B))
print('Number of units from vendor C: ', pyo.value(model.x_C))
print('Number of units from vendor D: ', pyo.value(model.x_D))
print('Number of units from vendor E: ', pyo.value(model.x_E))

Total Cost:  18892579.12212804
Number of units from vendor A:  33000000.329505075
Number of units from vendor B:  9147777.3767216
Number of units from vendor C:  165600001.65549955
Number of units from vendor D:  12000000.119417813
Number of units from vendor E:  19852700.518855963
